In [12]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [13]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test, edgeindex2adj
from models import GCN, GAT, LP
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')

In [14]:
# dataset = citeseer
# model = GCN(dataset.num_features, 24, dataset.num_classes)

dataset = cora
model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 8, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [15]:
ei = dataset[0].edge_index
adj = edgeindex2adj(ei, dataset.x.shape[0])

In [16]:
data = dataset[0]
data.train_mask[torch.randperm(data.train_mask.shape[0])[:int(0.9 * data.train_mask.shape[0])]] = False

best_val_acc = final_test_acc = 0
for epoch in range(1, 100):
    loss = train(model, data, optimizer, loss='cross_entropy')
    train_acc, val_acc, tmp_test_acc = test(model, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 1.954114317893982, Train: 0.2857, Val: 0.1580, Test: 0.1610
Epoch: 002, Loss: 1.8951189517974854, Train: 0.3571, Val: 0.1780, Test: 0.1680
Epoch: 003, Loss: 1.8434655666351318, Train: 0.4286, Val: 0.1780, Test: 0.1680
Epoch: 004, Loss: 1.785523533821106, Train: 0.4286, Val: 0.1780, Test: 0.1680
Epoch: 005, Loss: 1.7209957838058472, Train: 0.4286, Val: 0.1860, Test: 0.1840
Epoch: 006, Loss: 1.6529338359832764, Train: 0.5000, Val: 0.1900, Test: 0.1950
Epoch: 007, Loss: 1.5840550661087036, Train: 0.5000, Val: 0.2080, Test: 0.2020
Epoch: 008, Loss: 1.5144073963165283, Train: 0.6429, Val: 0.2160, Test: 0.2150
Epoch: 009, Loss: 1.4449814558029175, Train: 0.6429, Val: 0.2240, Test: 0.2280
Epoch: 010, Loss: 1.3775851726531982, Train: 0.6429, Val: 0.2340, Test: 0.2390
Epoch: 011, Loss: 1.313022255897522, Train: 0.6429, Val: 0.2400, Test: 0.2470
Epoch: 012, Loss: 1.2517119646072388, Train: 0.6429, Val: 0.2480, Test: 0.2550
Epoch: 013, Loss: 1.1939951181411743, Train: 0.7143, Va

In [6]:
# dataset = citeseer
# dataset = cora
dataset = pubmed

best_val_acc = final_test_acc = 0
best_l = best_a = 0

for l in [1, 2, 4, 8, 16, 32]:
    for a in [0.05, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.99, 1]:
        model = LP(num_layers=l, alpha=a)
        outs = model.train(dataset)
        train_acc, val_acc, tmp_test_acc = model.test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            best_l = l
            best_a = a
            
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}', f'Best l: {best_l}', f'Best a: {best_a}')

Best Val Acc: 0.7260 Test Acc: 0.7140 Best l: 16 Best a: 0.99


In [7]:
accs = model.test()
accs

[0.8833333333333333, 0.712, 0.707]

In [8]:
outs.shape

torch.Size([19717, 3])

In [9]:
outs.argmax(dim=-1)

tensor([1, 1, 0,  ..., 2, 0, 2])

In [10]:
dataset[0].y

tensor([1, 1, 0,  ..., 2, 0, 2])

In [11]:
dataset.num_classes

3